In [1]:
import pandas as pd

df = pd.DataFrame([
    [
        str(x),str(x%3),str(x//1000),f"ADE{str(x)[-1]}"
    ]
    for x in range(10000000)
])

El DataFrame que creamos vemos que al ser todo cadenas ocupa 305 MB

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
 1   1       object
 2   2       object
 3   3       object
dtypes: object(4)
memory usage: 305.2+ MB


Si usamos `memory_usage` vemos el consumo en bytes no solo de cada columna (que en este caso todas consumen lo mismo) sino que tambien del indice. El indice influye sobre el tamaño final del DataFrame, en general va a importar poco, pero si hacemos las cosas mal, va a consumir mas recursos

In [3]:
df.memory_usage()

Index         128
0        80000000
1        80000000
2        80000000
3        80000000
dtype: int64

Podemos ver que cambiando los elementos de la lista a int no nos cambia nada, porque sigue siendo una representacion de 64bits

In [4]:
df[0] = df[0].map(lambda x: int(x))
df[1] = df[1].astype(int)
df.memory_usage()

Index         128
0        80000000
1        80000000
2        80000000
3        80000000
dtype: int64

En numpy podemos encontrar tipos de datos mas especificos. Por ejemplo, para la segunda columna, al tener valores del 0 al 2, puedo representarla con un entero de 8bits

In [5]:
import numpy as pd
df[1] = df[1].astype(pd.int8)
df.memory_usage()

Index         128
0        80000000
1        10000000
2        80000000
3        80000000
dtype: int64

Notar que con este simple cambio, se redujo a 1/8 el uso de memoria de la columna. Elegir un tipo mas especifico tiene muchos beneficios

In [6]:
df[2] = df[2].astype(pd.int16)
df.memory_usage()

Index         128
0        80000000
1        10000000
2        20000000
3        80000000
dtype: int64

La cuarta columna por su parte, no puede ser representada como un tipo de dato entero, porque son cadenas. No obstante, si lo puedo convertir a una columna de tipo categoria, lo que genera practicamente el mismo efecto

In [7]:
df[3] = df[3].astype("category")
df[3]

0          ADE0
1          ADE1
2          ADE2
3          ADE3
4          ADE4
           ... 
9999995    ADE5
9999996    ADE6
9999997    ADE7
9999998    ADE8
9999999    ADE9
Name: 3, Length: 10000000, dtype: category
Categories (10, object): ['ADE0', 'ADE1', 'ADE2', 'ADE3', ..., 'ADE6', 'ADE7', 'ADE8', 'ADE9']

Notar que al ver la columna, ademas de los datos, ahora tenemos una tabla de traduccion. Esa tabla de traduccion tambien ocupa espacio. El uso lo podemos sacar de lo que nos muestra `memory_usage`. Al ser solo 10 valores, puede codificarlos en enteros de 8bits, por eso la columna deberia utilizar lo mismo que la segunda. Pero como tambien tiene que guardar el indice, utiliza 380 bytes mas

In [8]:
df.memory_usage()

Index         128
0        80000000
1        10000000
2        20000000
3        10000380
dtype: int64